In [1]:
import requests

def get_market_data(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        market_data = {
            "name": data.get("name"),
            "symbol": data.get("symbol"),
            "current_price_usd": data["market_data"]["current_price"].get("usd"),
            "market_cap_usd": data["market_data"]["market_cap"].get("usd"),
            "market_cap_rank": data.get("market_cap_rank"),
            "price_change_percentage_24h": data["market_data"].get("price_change_percentage_24h"),
            "total_volume_usd": data["market_data"]["total_volume"].get("usd")
        }
        return market_data
    else:
        raise Exception("Failed to fetch market data.")

def get_news_updates(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/status_updates"
    response = requests.get(url)

    if response.status_code == 200:
        updates = response.json().get("status_updates", [])
        news = [
            {
                "title": update.get("title"),
                "description": update.get("description")
            } for update in updates[:5]  # limit to 5 recent updates
        ]
        return news
    else:
        raise Exception("Failed to fetch updates.")


In [3]:
try:
        market = get_market_data("bitcoin")
        news = get_news_updates("bitcoin")

        print("=== Market Data ===")
        for k, v in market.items():
            print(f"{k}: {v}")

        print("\n=== Recent Updates ===")
        for update in news:
            print(f"- {update['title']}\n  {update['description']}\n")

except Exception as e:
        print(str(e))


Failed to fetch updates.


In [4]:
market = get_market_data("bitcoin")

In [5]:
market

{'name': 'Bitcoin',
 'symbol': 'btc',
 'current_price_usd': 105166,
 'market_cap_usd': 2089341145335,
 'market_cap_rank': 1,
 'price_change_percentage_24h': 2.12484,
 'total_volume_usd': 36807350634}

In [7]:
import pandas as pd
df = pd.DataFrame([market])

In [8]:
df

,name,symbol,current_price_usd,market_cap_usd,market_cap_rank,price_change_percentage_24h,total_volume_usd
0,Bitcoin,btc,105166,2089341145335,1,2.12484,36807350634


In [11]:
import requests
import csv
import os
from datetime import datetime
import pandas as pd
import json
import time


api_key = '419dc95e-f466-4c08-8c15-8873aa8511b0'
BASE_URL = "https://pro-api.coinmarketcap.com"
API_KEY = '419dc95e-f466-4c08-8c15-8873aa8511b0'
headers = {
    'X-CMC_PRO_API_KEY': api_key,
    'Accept': 'application/json',
}


def save_json(filename, data):
    """Save data to a JSON file."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"✅ Successfully saved data to {filename}")
    except Exception as e:
        print(f"❌ Error saving {filename}: {e}")


def fetch_data_from_api(url, params=None, max_retries=5, wait_time=10):
    """Fetch data from CoinMarketCap API with retry logic and rate limiting."""
    for attempt in range(max_retries):
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            return response.json()

        elif response.status_code == 429:
            print(f"⚠️ Rate limit reached. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)  # Wait before retrying

        else:
            print(f"❌ Error fetching data from {url}: {response.status_code} - {response.text}")
            break  # Exit loop on non-retryable errors

    return None  # Return None after max retries


def split_into_batches(data_list, batch_size=50):
    """Split a list into smaller batches to avoid URL length limits."""
    return [data_list[i:i + batch_size] for i in range(0, len(data_list), batch_size)]


In [ ]:
#from Imports import *


def fetch_cryptocurrency_listings_latest(params=None):
    """Fetch latest cryptocurrency listings."""
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
    return fetch_data_from_api(url, params)

def fetch_cryptocurrency_map():
    """Fetch cryptocurrency map with coin metadata."""
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/map"
    return fetch_data_from_api(url)




def main():
    # Fetch and save cryptocurrency map
    crypto_map = fetch_cryptocurrency_map()
    if crypto_map:
        with open('crypto_map.json', 'w') as f:
            json.dump(crypto_map, f, indent=4)
        print("Successfully saved cryptocurrency map to 'crypto_map.json'")

    crypto_listing = fetch_cryptocurrency_listings_latest()
    if crypto_listing:
        with open('crypto_listing.json', 'w') as f:
            json.dump(crypto_listing, f, indent=4)
        print("Successfully saved cryptocurrency listing to 'crypto_map.json'")



In [15]:
main()

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [23]:
import requests
import pandas as pd

def fetch_crypto_news():
    url = 'https://min-api.cryptocompare.com/data/v2/news/'

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        news_items = data.get('Data', [])

        # Prepare a list of dicts with relevant fields
        news_list = []
        for item in news_items:
            news_list.append({
                'title': item.get('title'),
                'source': item.get('source'),
                'url': item.get('url'),
                'published_on': item.get('published_on'),
                'categories': item.get('categories'),
                'language': item.get('lang')
            })

        # Create DataFrame
        df = pd.DataFrame(news_list)
        return df

    except requests.exceptions.RequestException as e:
        print("Error fetching crypto news:", e)
        return pd.DataFrame()  # return empty DataFrame on error

# Usage
df_news = fetch_crypto_news()
df_news

,title,source,url,published_on,categories,language
0,Altseason 2025: Bitcoin Dominance & Indicators...,coinpedia,https://coinpedia.org/price-analysis/altseason...,1747740287,ALTCOIN|BTC|ETH|MARKET|TRADING,EN
1,Analysts Suggest XRP Futures May Offer Opportu...,coinotag,https://en.coinotag.com/analysts-suggest-xrp-f...,1747740191,BTC|ETH|MARKET|TRADING|XRP,EN
2,Gate Alpha Lists MERL: Trade Popular Crypto As...,coinotag,https://en.coinotag.com/breakingnews/gate-alph...,1747739983,TRADING|USDT|BNB,EN
3,Caroline Crenshaw Warns SEC’s Crypto Rulebook ...,bitdegree,https://www.bitdegree.org/crypto/news/caroline...,1747739953,EXCHANGE|REGULATION,EN
4,South Korean Actress Hwang Jung-eum Admits to ...,cryptonews,https://cryptonews.com/news/south-korean-actre...,1747739783,ASIA|BTC|BUSINESS|REGULATION,EN
5,Binance Seeks Dismissal of FTX’s $1.76 Billion...,coinotag,https://en.coinotag.com/binance-seeks-dismissa...,1747739734,EXCHANGE|REGULATION|BNB|FTT,EN
6,Crypto Daybook Americas: Bitcoin Stability Abo...,coindesk,https://www.coindesk.com/daybook-us/2025/05/20...,1747739700,BTC|ETH|FIAT|MARKET|TRADING,EN
7,South Korea Eyes Won-Backed Stablecoin Ahead o...,coinpedia,https://coinpedia.org/news/south-korea-eyes-wo...,1747739633,ASIA|BTC|BUSINESS|MARKET|REGULATION,EN
8,Shiba Inu Battles Bearish Trends with Price Vo...,cointurken,https://en.coin-turk.com/shiba-inu-battles-bea...,1747739605,SHIB|TRADING,EN
9,Catena Labs Raises $18 Million Seed Round to R...,coinotag,https://en.coinotag.com/breakingnews/catena-la...,1747739590,OTHER,EN
